<p style="font-weight:bold;"> <span style="font-size: 36px"> Reimport of Transactional Data Test</span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

# Test Function

In [0]:
public async Task<ActivityLog> ReimportAndCompare((string transactionalData, string importFormat)[] importInfo, DataNodeState[] dataNodeStates)
{
    Activity.Start();
    var workspace = Workspace.CreateNew();
    workspace.Initialize(x => x.FromSource(DataSource)
                           .DisableInitialization<DataNodeState>());
    await workspace.UpdateAsync<DataNodeState>(dataNodeStates);

    foreach(var (transactionalData, importFormat) in importInfo)
        await Import.FromString(transactionalData).WithFormat(importFormat).WithTarget(workspace).ExecuteAsync();
    
    var originalRawVariables = workspace.Query<RawVariable>();
    var originalIfrsVariables = workspace.Query<IfrsVariable>();

    foreach(var (transactionalData, importFormat) in importInfo)
        await Import.FromString(transactionalData).WithFormat(importFormat).WithTarget(workspace).ExecuteAsync();

    var reimportedRawVariables = workspace.Query<RawVariable>();
    var reimportedIfrsVariables = workspace.Query<IfrsVariable>();

    List<string> errors = new();
    var missingRawVariables = originalRawVariables.Except(reimportedRawVariables);
    var missingIfrsVariables = originalIfrsVariables.Except(reimportedIfrsVariables);
    var extraRawVariables = reimportedRawVariables.Except(originalRawVariables);
    var extraIfrsVariables = reimportedIfrsVariables.Except(originalIfrsVariables);

    if(missingRawVariables.Any()) errors.Add($"Reimport missed the following RawVariables: {string.Join(", ", missingRawVariables)}.");
    if(missingIfrsVariables.Any()) errors.Add($"Reimport missed the following IfrsVariables: {string.Join(", ", missingIfrsVariables)}.");
    if(extraRawVariables.Any()) errors.Add($"Reimport created RawVariables that it shouldn't: {string.Join(", ", extraRawVariables)}.");
    if(extraIfrsVariables.Any()) errors.Add($"Reimport created IfrsVariables that it shouldn't: {string.Join(", ", extraIfrsVariables)}.");
    
    if(errors.Any()) ApplicationMessage.Log(Error.Generic, string.Join("\n", errors));

    ApplicationMessage.Log(Warning.Generic, "count of ifrs variables in workspace: "+reimportedIfrsVariables.Count());
    if (reimportedIfrsVariables.Any()) ApplicationMessage.Log(Warning.Generic, string.Join("-------------", reimportedIfrsVariables));

    return Activity.Finish();
}

# Test

## General data

In [0]:
var args = new ImportArgs("CH", 2021, 3, Periodicity.Quarterly, null, ImportFormats.Actual);
var previousArgs = new ImportArgs("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Actual);

var partitionReportingNode = new PartitionByReportingNode { Id = (Guid)(await DataSource.Partition.GetKeyForInstanceAsync<PartitionByReportingNode>(args)),
                                                        ReportingNode = args.ReportingNode};

In [0]:
var dt11StatePrevious = new DataNodeState {
            DataNode = "DT1.1",
            State = State.Active,
            Year = previousArgs.Year,
            Month = previousArgs.Month,
            Partition = partitionReportingNode.Id
        };

var dt11State = new DataNodeState {
            DataNode = "DT1.1",
            State = State.Active,
            Year = args.Year,
            Month = args.Month,
            Partition = partitionReportingNode.Id
        };

var dt13State = new DataNodeState {
            DataNode = "DT1.3",
            State = State.Active,
            Year = args.Year,
            Month = args.Month,
            Partition = partitionReportingNode.Id
        };

var dtr11StatePrevious = new DataNodeState {
            DataNode = "DTR1.1",
            State = State.Active,
            Year = previousArgs.Year,
            Month = previousArgs.Month,
            Partition = partitionReportingNode.Id
        };

var dtr11State = new DataNodeState {
            DataNode = "DTR1.1",
            State = State.Active,
            Year = args.Year,
            Month = args.Month,
            Partition = partitionReportingNode.Id
        };


In [0]:
DataSource.Query<IfrsVariable>().Count()

## Case: One Data Node at inception

In [0]:
var openings = 
@"@@Main
ReportingNode,Year,Month,,
CH,2020,12,,
@@Opening
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.1,C,,,72.2
DT1.1,AA,PR,,-1.5
DT1.1,OA,PR,,1.5
";

In [0]:
var importFormat = ImportFormats.Opening;
var activity = await ReimportAndCompare((openings, importFormat).RepeatOnce().ToArray(), dt11StatePrevious.RepeatOnce().ToArray());
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

## Case:  Actuals for one Data Node at inception and one not at inception

In [0]:
var actuals = @"
@@Main,,,,,
ReportingNode,Year,Month,,,
CH,2021,3,,,
@@Actual,,,,,
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,PR,A,,90
DT1.1,CF,ICO,A,,-6
DT1.1,CF,NIC,A,,-70
DT1.1,CF,ACA,A,,-10
DT1.1,CF,AEA,A,,-5
DT1.3,CF,PR,A,,90
DT1.3,CF,ICO,A,,-6
DT1.3,CF,NIC,A,,-70
DT1.3,CF,ACA,A,,-10
DT1.3,CF,AEA,A,,-5";

In [0]:
var importFormat = ImportFormats.Actual;
var activity = await ReimportAndCompare((actuals, importFormat).RepeatOnce().ToArray(), new[] {dt11State, dt11StatePrevious, dt13State});
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
activity

## Case: Openings for insurance at inception, and cashflows for linked reinsurance not at inception

In [0]:
var openings = 
@"@@Main
ReportingNode,Year,Month,,
CH,2021,3,,
@@Opening
DataNode,EstimateType,AmountType,AccidentYear,Value
DT1.1,C,,,72.2
DT1.1,AA,PR,,-1.5
DT1.1,OA,PR,,1.5
";

var cashflows = @"
@@Main
ReportingNode,Year,Month,
CH,2021,3,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20
DTR1.1,PR,BE,MC,I,,0,-55,0,0,-55,0,0,-55,0,0,-55,0,0,0,0,0,0,0,0,0,0
DTR1.1,NIC,BE,MC,I,,10,10,10,10,10,10,10,10,10,10,10,10,0,0,0,0,0,0,0,0,0
DTR1.1,,CU,MC,I,,-5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-1.5,0,0,0,0,0,0,0,0,0
DTR1.1,,RA,MC,I,,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0
DTR1.1,PR,BE,BOP,N,,0,-5,0,0,-5,0,0,-5,0,0,-5,0,0,0,0,0,0,0,0,0,0
DTR1.1,NIC,BE,BOP,N,,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.5,0,0,0,0,0,0,0,0,0
DTR1.1,,RA,BOP,N,,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0,0,0,0,0,0,0,0,0
DTR1.1,PR,BE,EV,N,,0,-2.5,0,0,-2.5,0,0,-2.5,0,0,-2.5,0,0,0,0,0,0,0,0,0,0
DTR1.1,NIC,BE,EV,N,,5,5,5,5,5,5,5,5,5,5,5,5,0,0,0,0,0,0,0,0,0
DTR1.1,,RA,EV,N,,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0,0,0,0,0,0,0,0,0
DTR1.1,PR,BE,CL,C,,0,-57.5,0,0,-57.5,0,0,-57.5,0,0,-57.5,0,0,0,0,0,0,0,0,0,0
DTR1.1,NIC,BE,CL,C,,15,15,15,15,15,15,15,15,15,15,15,15,0,0,0,0,0,0,0,0,0
DTR1.1,,CU,CL,C,,-7.5,-3.5,-3.5,-3.5,-3.5,-3.5,-3.5,-3.5,-3.5,-3.5,-3.5,-2.5,0,0,0,0,0,0,0,0,0
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,0,0,0,0,0,0,0,0,0";

In [0]:
var openingFormat = ImportFormats.Opening;
var cashflowFormat = ImportFormats.Cashflow;
var activity = await ReimportAndCompare(new[]{(openings, openingFormat), (cashflowFormat, cashflows)}, new[] {dt11State, dtr11StatePrevious, dtr11State});
activity

In [0]:
activity.Status.Should().Be(ActivityLogStatus.Succeeded);

In [0]:
DataSource.Query<IfrsVariable>().Count()